#### Laboratorio 2

##### Javier Valle 20159
##### Mario de León 19029

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization
from tensorflow.keras.optimizers import RMSprop



##### Analizando el dataset.

In [2]:
# Importando el dataset.
dataset = "dataset.csv"

# Cargando el dataset.
df = pd.read_csv(dataset)

# Imprimiendo el dataset.
df

# Imprimiendo los títulos del dataset.
df.columns

# Objetivo de la red: Clasificación.

# Posibles variables a utilizar para clasificar: 
# 1. Production budget $
# 2. Worldwide gross $
# 3. Domestic gross $


Index(['movie_title', 'production_date', 'genres', 'runtime_minutes',
       'director_name', 'director_professions', 'director_birthYear',
       'director_deathYear', 'movie_averageRating', 'movie_numerOfVotes',
       'approval_Index', 'Production budget $', 'Domestic gross $',
       'Worldwide gross $'],
      dtype='object')

##### Imprimiendo las columnas elegidas del dataset.

In [3]:
# Imprimiedo las columnas a utilizar (movie_averageRating, movie_numerOfVotes y approval_Index).

# Seleccionamos las columnas que queremos combinar
selected_columns = ['Production budget $', 'Worldwide gross $', 'Domestic gross $']

# Utilizamos la función concat() para combinar las columnas en una sola variable
combined_variable = pd.concat([df[col] for col in selected_columns], axis=1)

# Renombramos las columnas del nuevo DataFrame resultante
combined_variable.columns = ['movie_averageRating', 'movie_numerOfVotes', 'approval_Index']

# Ahora 'combined_variable' contiene todas las columnas seleccionadas combinadas en una sola variable
print(combined_variable)

      movie_averageRating  movie_numerOfVotes  approval_Index
0               460000000          2265935552       667830256
1               400000000          2794731755       858373000
2               379000000          1045713802       241071802
3               365000000          1395316979       459005868
4               300000000          2048359754       678815482
...                   ...                 ...             ...
4375                65000            22233808        11529368
4376                50000            10426506        10426506
4377                50000             2335352         2335352
4378                50000              424149          391674
4379                50000                8374            8374

[4380 rows x 3 columns]


##### Creando one-hot encoding - movie_title

In [4]:
# label -> one-hot encoding
label = pd.get_dummies(df['movie_title'], prefix='movie_title')
df = pd.concat([df, label], axis=1)
# drop old label
df.drop(['movie_title'], axis=1, inplace=True)

##### Creando un X y un Y para el entrenamiento.

In [5]:
# En la variable X se necesitaría meter esto 'Production budget $', 'Worldwide gross $', 'Domestic gross $'

X = df[['movie_averageRating', 'movie_numerOfVotes', 'approval_Index']]

# En el eje y sería esto 'movie_averageRating', 'movie_numerOfVotes', 'approval_Index'.
Y = df[['Production budget $', 'Worldwide gross $', 'Domestic gross $']]

##### Entrenando la red.

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X, 
    Y, 
    test_size=0.2, 
    )

# Verificando los resultados.
print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)
print("y_train: ", y_train.shape)
print("y_test: ", y_test.shape)

X_train:  (3504, 3)
X_test:  (876, 3)
y_train:  (3504, 3)
y_test:  (876, 3)


##### Creando la red

In [7]:
# Nueva arquitectura ajustada
batched_model = Sequential([
    Dense(128, input_shape=(3,), activation="relu"),   # Aumentamos el número de neuronas a 128
    BatchNormalization(),
    Dense(256, activation='relu'),                      # Aumentamos el número de neuronas a 256
    BatchNormalization(),
    Dense(256, activation='relu'),                      # Aumentamos el número de neuronas a 256
    BatchNormalization(),
    Dense(128, activation='relu'),                      # Ajustamos las neuronas a 128
    BatchNormalization(),
    Dense(64, activation='relu'),                       # Ajustamos las neuronas a 64
    BatchNormalization(),
    Dense(3, activation='softmax')
])

In [ ]:
# Compilar el modelo con RMSprop como optimizador
batched_model.compile(
    optimizer=RMSprop(), 
    loss='mean_squared_error', 
    metrics=['accuracy']
)

In [ ]:
batched_history =  batched_model.fit(
    X_train, 
    y_train,
    epochs=100, 
    validation_split=0.25,
    batch_size=500,
    verbose=2
)

Epoch 1/100
6/6 - 4s - loss: 18695728498999296.0000 - accuracy: 0.4471 - val_loss: 27731059789529088.0000 - val_accuracy: 0.7466 - 4s/epoch - 645ms/step
Epoch 2/100
6/6 - 0s - loss: 18695728498999296.0000 - accuracy: 0.6233 - val_loss: 27731059789529088.0000 - val_accuracy: 0.7466 - 303ms/epoch - 51ms/step
Epoch 3/100
6/6 - 0s - loss: 18695728498999296.0000 - accuracy: 0.7066 - val_loss: 27731059789529088.0000 - val_accuracy: 0.7489 - 291ms/epoch - 49ms/step
Epoch 4/100
6/6 - 0s - loss: 18695728498999296.0000 - accuracy: 0.7215 - val_loss: 27731059789529088.0000 - val_accuracy: 0.6998 - 285ms/epoch - 48ms/step
Epoch 5/100
6/6 - 0s - loss: 18695728498999296.0000 - accuracy: 0.7207 - val_loss: 27731059789529088.0000 - val_accuracy: 0.7409 - 281ms/epoch - 47ms/step
Epoch 6/100
6/6 - 0s - loss: 18695728498999296.0000 - accuracy: 0.7237 - val_loss: 27731059789529088.0000 - val_accuracy: 0.7374 - 297ms/epoch - 50ms/step
Epoch 7/100
6/6 - 0s - loss: 18695728498999296.0000 - accuracy: 0.7348 -

6/6 - 0s - loss: 18695730646482944.0000 - accuracy: 0.7542 - val_loss: 27731059789529088.0000 - val_accuracy: 0.7055 - 376ms/epoch - 63ms/step
Epoch 95/100
6/6 - 0s - loss: 18695728498999296.0000 - accuracy: 0.7538 - val_loss: 27731059789529088.0000 - val_accuracy: 0.7192 - 314ms/epoch - 52ms/step
Epoch 96/100
6/6 - 0s - loss: 18695728498999296.0000 - accuracy: 0.7538 - val_loss: 27731059789529088.0000 - val_accuracy: 0.7203 - 321ms/epoch - 53ms/step
Epoch 97/100
6/6 - 0s - loss: 18695730646482944.0000 - accuracy: 0.7519 - val_loss: 27731059789529088.0000 - val_accuracy: 0.7100 - 346ms/epoch - 58ms/step
Epoch 98/100
6/6 - 0s - loss: 18695728498999296.0000 - accuracy: 0.7515 - val_loss: 27731059789529088.0000 - val_accuracy: 0.7078 - 358ms/epoch - 60ms/step
Epoch 99/100
6/6 - 0s - loss: 18695730646482944.0000 - accuracy: 0.7451 - val_loss: 27731059789529088.0000 - val_accuracy: 0.7192 - 317ms/epoch - 53ms/step
Epoch 100/100
6/6 - 0s - loss: 18695728498999296.0000 - accuracy: 0.7576 - va